In [ ]:
import urllib.request

def make_data_object_name(dataset_name, year, month, day, hour, realization, forecast_period):
    template_string = "prods_op_{}_{:02d}{:02d}{:02d}_{:02d}_{:02d}_{:03d}.nc"
    return template_string.format(dataset_name, year, month, day, hour, realization, forecast_period)

def download_data_object(dataset_name, data_object_name, parent):
    url = "https://s3.eu-west-2.amazonaws.com/" + dataset_name + "/" + data_object_name
    urllib.request.urlretrieve(url, parent + '/' + data_object_name) # save in this directory with same name

In [ ]:
import os
import boto3
from botocore.client import Config

def boto_download(prefix, download_dir='./'):
    session = boto3.session.Session(aws_access_key_id=os.environ['ACCESS_KEY'], # These need to be set at environmental
                                    aws_secret_access_key=os.environ['SECRET_KEY']) # variables as per your AWS credentials
    s3 = session.resource('s3', config=Config(signature_version='s3v4'))
    bucket = s3.Bucket('mogreps-uk')
    keys = bucket.objects.filter(Prefix=prefix)
    for key in keys:
        key_name = key.key
        fpath = '{}{}'.format(download_dir, key_name)
        if os.path.isfile(fpath):
            print('{} already exists, skipping...'.format(fpath))
        else:
            print('Downloading {}...'.format(key_name))
            with open(fpath, 'wb') as outf:
                bucket.download_fileobj(Key=key_name, Fileobj=outf)

In [ ]:
for month in [7, 8, 9]:
    for day in [1, 4, 7, 15, 18, 21]:
        for hour in [3, 9, 15, 21]:
            for fcst in [3, 6]:
                boto_download('prods_op_mogreps-uk_2016{:02d}{:02d}_{:02d}_00_{:03d}'.format(month, day, hour, fcst),
                              '../data/')